In [1]:
# Libraries import

import pandas as pd
import numpy as np
import re
import pickle

In [2]:
# Logs file

# Settings
log_columns = ["Time", "IP", "Browser", "GA Value", "User ID", "Transaction ID"]

# Reading the CSV file / constructing the Data Frame
logs = pd.read_csv('dataset3/log.tsv', sep="\t", header=None)
logs.columns = log_columns
logs["Logs"] = True



# Specific processinf for Paris en Scene

logs["Transaction ID"] = [x[:6] for x in logs["Transaction ID"]]
logs["Transaction ID"] = logs["Transaction ID"].astype(int)
logs = logs.drop_duplicates(subset="Transaction ID")
logs = logs.reset_index(drop=True)

In [3]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime

dates = pd.to_datetime(logs['Time'])
mydf = pd.DataFrame(logs['Logs'])
mydf = mydf.set_index(dates)

ddf = mydf.resample('D').sum()
ax = ddf.plot(figsize=(18,5), kind='bar', title='Transactions par jour', rot=45)
ax.set(xlabel="Jour")
ax.xaxis.set_ticklabels([x.strftime('%d/%m/%Y') for x in ddf.index])

hdf = mydf.groupby(mydf.index.hour).sum()

hdf.plot(figsize=(18,5), kind='bar', title='Transactions par heure', rot=0)

In [4]:
# Back Office extract

# Settings
bo_columns = ["Transaction ID"]

# Reading the CSV File / Constructing the Data Frame
bo = pd.read_csv('dataset3/bo.tsv', sep="\t")
bo.columns = bo_columns
bo["BO"] = True

# Specific processinf for Paris en Scene

bo["Transaction ID"] = [str(x)[:6] for x in bo["Transaction ID"]]
bo["Transaction ID"] = bo["Transaction ID"].astype(int)
bo = bo.drop_duplicates(subset="Transaction ID")
bo = bo.reset_index(drop=True)

In [5]:
# Google Analytics Extract

# Settings
ga_columns = ["Transaction ID"]

# Reading the CSV File / Constructing the Data Frame
ga = pd.read_csv('dataset3/ga.tsv', sep="\t")
ga.columns = ga_columns
ga["GA"] = True

# Specific processinf for Paris en Scene

ga["Transaction ID"] = [x[:6] for x in ga["Transaction ID"]]
ga["Transaction ID"] = ga["Transaction ID"].astype(int)
ga = ga.drop_duplicates(subset="Transaction ID")
ga = ga.reset_index(drop=True)

In [6]:
# Merging the three sources : Logs / GA / BO
def isCookie(cookie):
    """ Check if the value is a GA Cookie """
    my_regex = re.compile(r"^GA1")
    
    if (type(cookie) != str):
        return False
    
    elif (my_regex.search(cookie) is None):
        return False

    else:
        return True

# Merging
merged_df = logs.merge(bo, how='outer', on="Transaction ID").fillna(False)
merged_df = merged_df.merge(ga, how='outer', on="Transaction ID").fillna(False)

# Addding the Cookie Value Check
merged_df["Cookie"] = [isCookie(x) for x in merged_df["GA Value"]]

In [7]:
# Generating the Results CSV File

merged_df.to_csv("results.csv", sep=',', encoding="utf-8")

In [8]:
# Preparing the results variables

just_logs = len(merged_df[merged_df["Logs"] == True].index)
just_bo = len(merged_df[merged_df["BO"] == True].index)
just_ga = len(merged_df[merged_df["GA"] == True].index)
just_cookie = len(merged_df[merged_df["Cookie"] == True].index)

bo_and_ga = len(merged_df[(merged_df["BO"] == True) & (merged_df["GA"] == True)].index)
bo_without_ga = len(merged_df[(merged_df["BO"] == True) & (merged_df["GA"] == False)].index)
bo_without_ga_with_cookie = len(merged_df[(merged_df["BO"] == True) & (merged_df["GA"] == False)  & (merged_df["Cookie"] == True)].index)
bo_without_ga_without_cookie = len(merged_df[(merged_df["BO"] == True) & (merged_df["GA"] == False)  & (merged_df["Cookie"] == False)].index)
bo_and_logs = len(merged_df[(merged_df["BO"] == True) & (merged_df["Logs"] == True)].index)
bo_without_logs = len(merged_df[(merged_df["BO"] == True) & (merged_df["Logs"] == False)].index)

bo_and_logs_and_cookie = len(merged_df[(merged_df["BO"] == True) & (merged_df["Logs"] == True) & (merged_df["Cookie"] == True)].index)
bo_and_logs_and_cookie_and_ga = len(merged_df[(merged_df["BO"] == True) & (merged_df["Logs"] == True) & (merged_df["Cookie"] == True) & (merged_df["GA"] == True)].index)
bo_and_logs_and_cookie_without_ga = len(merged_df[(merged_df["BO"] == True) & (merged_df["Logs"] == True) & (merged_df["Cookie"] == True) & (merged_df["GA"] == False)].index)
logs_and_cookie = len(merged_df[(merged_df["Logs"] == True) & (merged_df["Cookie"] == True)].index)
logs_without_cookie = len(merged_df[(merged_df["Logs"] == True) & (merged_df["Cookie"] == False)].index)

In [9]:
# Synthesis

print("")
print("-----> Transactions en Back Office : {} ({}%)".format(just_bo, int(just_bo/just_bo*100)))
print("-----> Présent dans les logs : {} ({}%)".format(bo_and_logs, round(bo_and_logs/just_bo*100, 1)))
print("-----> Et avec cookie : {} ({}%)".format(bo_and_logs_and_cookie, round(bo_and_logs_and_cookie/bo_and_logs*100, 1)))
print("-----> Et dans GA : {} ({}%)".format(bo_and_logs_and_cookie_and_ga, round(bo_and_logs_and_cookie_and_ga/bo_and_logs_and_cookie*100, 1)))
print("-----> Non dans GA : {} ({}%)".format(bo_and_logs_and_cookie_without_ga, round(bo_and_logs_and_cookie_without_ga/bo_and_logs_and_cookie*100, 1)))
print("")
print("-----> Absent dans les logs : {} ({}%)".format(bo_without_logs, int(bo_without_logs/just_bo*100)))
print("")
print("-----> Avec cookie : {} ({}%)".format(bo_without_ga_with_cookie, int(bo_without_ga_with_cookie/just_cookie*100)))
print("-----> Sans cookie : {} ({}%)".format(bo_without_ga_without_cookie, int(bo_without_ga_without_cookie/just_bo*100)))
print("")


-----> Transactions en Back Office : 154 (100%)
-----> Présent dans les logs : 122 (79.2%)
-----> Et avec cookie : 114 (93.4%)
-----> Et dans GA : 105 (92.1%)
-----> Non dans GA : 9 (7.9%)

-----> Absent dans les logs : 32 (20%)

-----> Avec cookie : 9 (7%)
-----> Sans cookie : 38 (24%)

